## Fact Table Modeling

In [0]:
last_gold_insert_date = spark.sql("select coalesce(max(last_modified_date), timestamp('1900-01-01')) as last_dt from databricks_catalog.gold.fact_complaint")
last_gold_insert_date.display()

In [0]:
from pyspark.sql.functions import col

# spark.sql("select * from databricks_catalog.silver.complaints").withColumn('last_insert_date', col('_metadata.file_modification_time'))

stage_silver_latest_df = spark.sql("select * from databricks_catalog.silver.complaints").filter(col('_metadata.file_modification_time') > last_gold_insert_date.select('last_dt').collect()[0][0])
stage_silver_latest_df.display()

In [0]:
stage_silver_latest_df.createOrReplaceTempView('stage_silver_latest')

In [0]:
latest_fact_df = spark.sql("""
          SELECT 
            sc.complaint_id AS complaint_id,
            dc.company_id AS company_id,
            ddr.date_id AS date_received_id,
            dsc.date_id AS date_sent_to_company_id,
            dcr.response_id AS response_id, 
            dd.demographic_id AS demographic_id,
            di.issue_id AS issue_id,
            dp.category_id AS product_id,
            sc.consumer_complaint_narrative AS consumer_complaint_narrative,
            sc.consumer_consent_provided AS consumer_consent_provided,
            sc.submitted_via AS submitted_via,
            sc.timely_response AS timely_response,
            sc.Resolved AS resolved,
            sc.complaint_age AS complaint_age,
            current_timestamp() AS last_modified_date
          FROM stage_silver_latest sc
            LEFT OUTER JOIN databricks_catalog.gold.dim_company dc ON sc.company = dc.company_name
            LEFT OUTER JOIN databricks_catalog.gold.dim_company_response dcr ON sc.company_public_response = dcr.company_public_response AND sc.company_response_to_consumer = dcr.company_response_to_consumer
            LEFT OUTER JOIN databricks_catalog.gold.dim_date ddr ON sc.date_received = ddr.origin_date
            LEFT OUTER JOIN databricks_catalog.gold.dim_date dsc ON sc.date_sent_to_company = dsc.origin_date
            LEFT OUTER JOIN databricks_catalog.gold.dim_demographic dd ON sc.state = dd.state AND sc.zip_code = dd.zip_code
            LEFT OUTER JOIN databricks_catalog.gold.dim_issue_category di ON sc.issue = di.issue AND sc.sub_issue = di.sub_issue
            LEFT OUTER JOIN databricks_catalog.gold.dim_product_category dp ON sc.product = dp.product AND sc.sub_product = dp.sub_product
          """)




Databricks data profile. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import col, sum as spark_sum

id_columns = ['complaint_id', 'company_id', 'date_received_id', 'date_sent_to_company_id', 'response_id', 'demographic_id', 'issue_id', 'product_id']
null_percent_df = latest_fact_df.select([(spark_sum(col(c).isNull().cast("int")) / latest_fact_df.count()).alias(c) for c in id_columns])
display(null_percent_df)

In [0]:
latest_fact_df.filter(col('demographic_id').isNull()).display()

In [0]:
facts_df = spark.sql("select * from databricks_catalog.gold.fact_complaint")

In [0]:
from delta.tables import DeltaTable
DeltaTable.forPath(spark, 'abfss://gold@rcmadls10dev.dfs.core.windows.net/fact_complaint').alias('target').merge(
    latest_fact_df.alias('source'),
    'target.complaint_id = source.complaint_id'
).whenNotMatchedInsertAll().execute()